# The improper dihedral for benzene does not get written by ParmEd

This is a problem if someone wants to parameterize a system with SMIRNOFF99Frosst, and use ParmEd to save a `.frcmod` file so that AmberTools (e.g., `tleap`) can be used downstream to add solvent or otherwise manipulate the structure.

In [1]:
import numpy as np
import simtk

This is using the 0.4.1 version of the toolkit.

In [2]:
import openforcefield

In [3]:
openforcefield.__version__

'0.4.1'

The starting point is a standard `.mol2` file with Tripos atom names. (OpenEyes only)

## AMBER workflow to create `.frcmod` file for benzene (for example)

Let's create an AMBER `frcmod` for reference:


```bash
antechamber -fi mol2 -i benzene.tripos.mol2 -fo mol2 -o benzene.gaff.mol2
tleap -f tleap.gaff.in
parmchk2 -i benzene.gaff.mol2 -f mol2 -o benzene.gaff.frcmod -s 2 -a Y
```

where `tleap.gaff.in` is:
```
source leaprc.gaff2
mol = loadmol2 benzene.gaff.mol2
saveamberparm mol benzene.gaff.prmtop benzene.gaff.inpcrd
```

The contents of the GAFF2 `frcmod` file is:

```
Remark line goes here
MASS
ca 12.010        0.360
ha 1.008         0.135

BOND
ca-ca  378.57   1.398
ca-ha  395.72   1.086

ANGLE
ca-ca-ca   68.767     120.020
ca-ca-ha   48.680     119.880

DIHE
ca-ca-ca-ca   4   14.500       180.000           2.000
ca-ca-ca-ha   4   14.500       180.000           2.000
ha-ca-ca-ha   4   14.500       180.000           2.000

IMPROPER
ca-ca-ca-ha         1.1          180.0         2.0          Using general improper torsional angle  X- X-ca-ha, penalty score=  6.0)

NONBON
  ca          1.8606  0.0988
  ha          1.4735  0.0161

```

## `openforcefield` toolkit workflow

Tripols MOL2 → OEMols → OpenMM system → MOL2, PRMTOP, FRCMOD (via ParmED)

In [4]:
from openforcefield.typing.engines.smirnoff import ForceField
from openforcefield.topology import Molecule, Topology

Unable to load toolkit <openforcefield.utils.toolkits.OpenEyeToolkitWrapper object at 0x7f1e001c47f0>.


In [5]:
import parmed as pmd

### 1. Read in `.mol2`

In [6]:
mol = openforcefield.topology.Molecule.from_smiles('c1ccccc1')
mol.generate_conformers()

In [7]:
mol

Molecule with name '' and SMILES '[H][C]1=[C]([H])[C]([H])=[C]([H])[C]([H])=[C]1[H]'

### 2. Create topology

In [8]:
topology = openforcefield.topology.Topology.from_molecules([mol])

In [9]:
print(f"I see {len([i for i in topology.topology_atoms])} atoms.")


print(f"I see {len([i for i in topology.topology_bonds])} bonds.")
print(f"I see {topology.n_angles} angles.")
print(f"I see {topology.n_propers} dihedrals.")
print(f"I see {topology.n_impropers} impropers.")

I see 12 atoms.
I see 12 bonds.
I see 18 angles.
I see 24 dihedrals.
I see 36 impropers.


### 3. Parameterize

In [10]:
ff = ForceField("test_forcefields/smirnoff99Frosst.offxml")

In [11]:
system = ff.create_openmm_system(topology)

In [12]:
print(f"I see {system.getNumParticles()} atoms.")

for force in system.getForces():
    if isinstance(force, simtk.openmm.openmm.HarmonicBondForce):
        print(f"I see {force.getNumBonds()} bonds.")
    if isinstance(force, simtk.openmm.openmm.HarmonicAngleForce):
        print(f"I see {force.getNumAngles()} angles.")
    if isinstance(force, simtk.openmm.openmm.PeriodicTorsionForce):
        print(f"I see {force.getNumTorsions()} mixed proper and improper dihedrals.")

I see 12 atoms.
I see 42 mixed proper and improper dihedrals.
I see 18 angles.
I see 12 bonds.


### 4. Bring into ParmEd

As noted by Rafal, ParmEd thinks that the central atom in an improper is the third one, but in OpenForceField is the first one! Patch that function.

In [34]:
# Monkey patch _process_dihedral to fix getting the first atom!
from parmed.topologyobjects import (Angle, AngleType, Atom, AtomType, Bond, BondType, Cmap, CmapType,
                               Dihedral, DihedralType, ExtraPoint, Improper, ImproperType,
                               NonbondedException, NonbondedExceptionType, RBTorsionType,
                               UreyBradley)
def _process_dihedral(struct, force):
    """ Adds periodic torsions to the structure """
    typemap = dict()
    for ii in range(force.getNumTorsions()):
        i, j, k, l, per, phase, phi_k = force.getTorsionParameters(ii)
        ai, aj = struct.atoms[i], struct.atoms[j]
        ak, al = struct.atoms[k], struct.atoms[l]
        key = (per, phase._value, phi_k._value)
        if key in typemap:
            dihed_type = typemap[key]
        else:
            dihed_type = DihedralType(phi_k, per, phase)
            typemap[key] = dihed_type
            struct.dihedral_types.append(dihed_type)
        # This is the line that has been changed (ak.bond_partners -> ai.bond_partners)
        improper = (ak in ai.bond_partners and aj in ai.bond_partners and
                    al in ai.bond_partners)
        struct.dihedrals.append(Dihedral(ai, aj, ak, al, improper=improper,
                                         type=dihed_type))
    struct.dihedral_types.claim()
pmd.openmm.topsystem._process_dihedral_old = pmd.openmm.topsystem._process_dihedral
pmd.openmm.topsystem._process_dihedral = _process_dihedral

In [35]:
parmed_structure = pmd.openmm.load_topology(topology.to_openmm(), system)

In [36]:
print(f"I see {len(parmed_structure.atoms)} atoms.")


print(f"I see {len(parmed_structure.bonds)} bonds.")
print(f"I see {len(parmed_structure.angles)} angles.")
print(f"I see {len(parmed_structure.dihedrals)} mixed proper and improper dihedrals.")

I see 12 atoms.
I see 12 bonds.
I see 18 angles.
I see 42 mixed proper and improper dihedrals.


In ParmEd, I think propers and impropers are all in the same list. Only CHARMM-style impropers are included in `Structure.impropers`. See this comment: https://github.com/ParmEd/ParmEd/issues/990#issuecomment-397844139

We can directly query the distinct propers and impropers, though.

In [37]:
parmed_structure.dihedral_types

TrackedList([
	<DihedralType; phi_k=0.367, per=2, phase=180.000,  scee=1.000, scnb=1.000>
	<DihedralType; phi_k=3.625, per=2, phase=180.000,  scee=1.000, scnb=1.000>
])

**Potential bug 1. These dihedral scaling factors are wrong / not the ones intended.**

I'm almost positive these should be `scee=1.2` (i.e., 1.0/1.2 = 0.83333...) and `scnb=2.0` (i.e., 1.0/2.0 = 0.5) for the 1-4 interactions.

In [38]:
parmed_structure.improper_types

TrackedList([
])

Note the complete lack of impropers here.

In [39]:
parmed_structure.save("../data/benzene.smirnoff.prmtop", overwrite=True)

In [40]:
!cat ../data/benzene.smirnoff.prmtop

%VERSION  VERSION_STAMP = V0001.000  DATE = 07/12/19  13:07:53
%FLAG TITLE
%FORMAT(20a4)

%FLAG POINTERS
%FORMAT(10I8)
      12       2       6       6      12       6      36       6       0       0
      52       1       6       6       6       2       2      23       1       0
       0       0       0       0       0       0       0       0      12       0
       0
%FLAG ATOM_NAME
%FORMAT(20a4)
                                                
%FLAG CHARGE
%FORMAT(5E16.8)
 -2.36889900E+00 -2.36889900E+00 -2.36889900E+00 -2.36889900E+00 -2.36889900E+00
 -2.36889900E+00  2.36889900E+00  2.36889900E+00  2.36889900E+00  2.36889900E+00
  2.36889900E+00  2.36889900E+00
%FLAG ATOMIC_NUMBER
%FORMAT(10I8)
       6       6       6       6       6       6       1       1       1       1
       1       1
%FLAG MASS
%FORMAT(5E16.8)
  1.20107800E+01  1.20107800E+01  1.20107800E+01  1.20107800E+01  1.20107800E+01
  1.20107800E+01  1.00794700E+00  1.00794700E+00  1.00794700E+00  1.00794700E+00
  1.0

**Potential bug 2. We can't yet write a `.frcmod` because the atoms do not have names.**  There probably also should have been an error raised for writing the `.prmtop`

In [31]:
for index, atom in enumerate(parmed_structure.atoms):
    print(index, atom.name, atom.type)

0  C1
1  C1
2  C1
3  C1
4  C1
5  C1
6  H1
7  H1
8  H1
9  H1
10  H1
11  H1


**Potential bug 3. We cannot write out a `.frcmod`** This error looks new to me. Not sure yet how to debug.

In [32]:
for index, atom in enumerate(parmed_structure.atoms):
    atom.name = index
    
parm_set = pmd.amber.AmberParameterSet.from_structure(parmed_structure)
parm_set.write("../data/benzene.smirnoff.frcmod")

In [33]:
!cat ../data/benzene.smirnoff.frcmod

Created by ParmEd
MASS
C1    12.011
H1     1.008

BOND
C1-C1    469.000   1.400
C1-H1    367.000   1.080

ANGLE
C1-C1-C1     70.000  120.000
C1-C1-H1     50.000  120.000

DIHE
C1-C1-C1-C1    1     3.62500000  180.000   2.0    SCEE=1.0 SCNB=1.0
C1-C1-C1-H1    1     3.62500000  180.000   2.0    SCEE=1.0 SCNB=1.0
H1-C1-C1-H1    1     3.62500000  180.000   2.0    SCEE=1.0 SCNB=1.0

IMPROPER
C1-C1-C1-H1     0.36666667  180.000   2.0

NONB
C1    1.90800000   0.08600000
H1    1.45900000   0.01500000



Success - Impropers is now populated!

**Potential bug 4.** Even if a `.frcmod` could be written, the `IMPROPER` section would be empty, which is wrong. I'm not sure if (a) they get written to the `DIHEDRAL` section and get silently ignored by AMBER (at best undefined behavior, as far as I can tell) or (b) they are just not written at all. This may be a bug upstream of ParmEd, as it looks like the OpenMM `System` also doesn't have the correct number of torsions. I'm not 100% sure I understand what is going on here.

In [41]:
!conda list

# packages in environment at /home/jaime/.local/anaconda/envs/openforcefield:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
alabaster                 0.7.12                     py_0    conda-forge
ambermini                 16.16.0                       7    omnia
asn1crypto                0.24.0                py37_1003    conda-forge
atomicwrites              1.3.0                      py_0    conda-forge
attrs                     19.1.0                     py_0    conda-forge
babel                     2.6.0                      py_1    conda-forge
backcall                  0.1.0                      py_0    conda-forge
bleach                    3.1.0                      py_0    conda-forge
blosc                     1.16.3               hf484d3e_0    conda-forge
bokeh                     1.1.0                    py37_0    conda-forge
boost                     1.68.0          py37h8619c78_1001    conda-